# Segmentación basada en grafos
Universidad de Chile  
Departamento de Ciencias de la Computación  
CC5508 - Procesamiento y Análisis de Imágenes  
Gabriel De La Parra  

## Introducción
Esta tarea se basa en la implementación del paper *[Efficient Graph-Based Image Segmentation](https://cs.brown.edu/~pff/papers/seg-ijcv.pdf)* de *[Pedro F. Felzenzswalb](https://cs.brown.edu/~pff/)*. 

##### Imágen como grafo
En la investigación se expone una técnica de *[segmentación de imagenes](https://en.wikipedia.org/wiki/Image_segmentation)* utilizando un *[grafo no dirigido](https://es.wikipedia.org/wiki/Grafo#Grafo_no_dirigido)* para representar la imagen de entrada:
* Los nodos del grafo representan los pixeles de la imagen. 
* Los arcos del grafo representan la relación entre pixeles vecinos.
* A cada arco se le asocia un peso. De lo anterior, cada peso determina la relación entre pixeles vecinos.

##### Segmentación
La segmentación consiste en generar conjuntos de nodos (o componentes conexas).

Para lo anterior se definen dos métricas:
* *Diferencia interna de una componente conexa*.
* *Diferencia entre dos componentes conexas*. 

Lo anterior pretende agrupar pixeles en regiones que sean similares entre sí y diferentes a otras regiones.

##### Proceso de la segmentación
Para la segmentación se procede de la siguiente manera:
* Con las métricas anteriores, se itera sobre las fronteras de cada componente.
* Sobre dicha frontera se busca evaluar si (los pesos de) la *diferencia entre las componentes* es suficientemente mayor a (los pesos de) la *diferencia interna* (de cada componente).
* A partir de esta evaluación se puede generar una única región o mantener las dos regiones.

Para mejorar este proceso se define un umbral sobre la evaluación de las diferencias.

Lo anterior se realiza para todos los arcos del grafo.

##### Métricas para la segmentación
La *diferencia entre dos componentes* se define como el peso mínimo a lo largo de (los arcos de) la frontera entre los dos grupos de nodos.

Por su parte, *la diferencia interna*, se define como el peso máximo en el MST de la componente.

Un MST (*[minumum spanning tree](https://en.wikipedia.org/wiki/Minimum_spanning_tree)* o *[árbol cobertor mínimo](https://es.wikipedia.org/wiki/%C3%81rbol_recubridor_m%C3%ADnimo)*) es una estructura de datos que permite ordenar todos los nodos de un grafo.

[![Minimum spanning tree](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d2/Minimum_spanning_tree.svg/300px-Minimum_spanning_tree.svg.png)](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d2/Minimum_spanning_tree.svg/300px-Minimum_spanning_tree.svg.png)

> Un ejemplo de árbol recubridor mínimo. Cada punto representa un vértice, cada arista está etiquetada con su peso, que en este caso equivale a su longitud. - Wikipedia

#### Particularidades de la tarea
Para esta tarea se deben tomar ciertas particularidades:

> Considerando que el algoritmo se basa en el método de [Kruskal](https://es.wikipedia.org/wiki/Algoritmo_de_Kruskal) para obtener árboles cobertores mínimos, será necesario ocupar un algoritmo eficiente para determinar:

> i. Los componentes que conectan una arista (find) 

> ii. Mezclar o unir dos componentes (union). 

> Para este fin el alumno deberá utilizar el algoritmo Union-Find que permite realizar la operación de find y union en un tiempo casi constante. 

#### Union-Find
[Union-Find](https://en.wikipedia.org/wiki/Disjoint-set_data_structure) es un algoritmo para estructuras de datos disjuntas. *Una estructura de datos para conjuntos disjuntos, es una estructura de datos que mantiene un conjunto de elementos particionados en un número de conjuntos disjuntos (no se solapan los conjuntos).* 

*Union*: Une dos subconjuntos en uno solo.

*Find*: Determina a cual subconjunto pertenece un elemento. Esta operación puede usarse para verificar si dos elementos están en el mismo conjunto.

La [complejidad del algoritmo](https://en.wikipedia.org/wiki/Disjoint-set_data_structure#Time_Complexity) se diferencia por 4 formas de implementación:
* *Union* con *Union-by-rank*
* *Find* con *path compression*
* Ambas
* Ninguna

*Union-by-rank*, consiste en añadir el árbol más pequeño a la raíz del árbol más grande. Como la profundidad del árbol afecta el tiempo de ejecución del algoritmo, el árbol con menor profundidad es añadido a la raíz del árbol con mayor profundidad, el cual aumenta su profundidad solo si sus profundidades son iguales.

*Find* con *path compression*, es una forma recursiva de aplanar la estructura del árbol. Cada nodo visitado en el camino hacia la raíz se añade directamente a la raíz ya que todos estos nodos la comparten. El árbol resultante es más aplanado, acelerando operaciones futuras no solo en estos elementos sino también en aquellos que referencian a estos.

## Desarrollo

##### Detalles del algoritmo
Para la segmentación se propone el siguiente algoritmo:
1. Ordenar los arcos por sus pesos, de menor a mayor.
2. Definir una segmentación inicial, en la que cada primer nodo es una componente por si sola.
3. Para cada arco:
    * Construir una componente nueva a partir de la componente anterior:
        * Si los dos nodos no pertenecen a la misma componente anterior y el peso entre ellos  es bajo, comparado con la diferencia entre las dos componentes, unir las componentes. Si no, continua.
4. Repetir 3. para todos los arcos del grafo.


#### Aplicaciones de la segmentación (Conc)
La investigación parte con la necesidad de tener algoritmos eficientes para segmentación de imágenes. Algunas de las aplicaciones que se mencionan son los problemas de dificultad media como reconocimiento estéreo, en donde son necesarias definir  *region of support* para las operaciones de correspondencia. Dichas regiones pueden ser encontradas con algoritmos de segmentación. Los problemas de alto nivel también pueden beneficiarse con algoritmos que sirvan para diferenciar el fondo de los objectos y el reconocimiento de las distintas partes.


#### Características deseadas de la segmentación (Conc)
Según el autor, un algoritmo de segmentación debe ser capaz de: 
1. *Capturar regiones o grupos de importancia perceptual que reflejan aspectos globales de la imagen*. Sobre esto, dos problemas principales son poder caracterizar aquello que es importante de la imagen y especificar que es lo que hace una determinada técnica de segmentación.
2. *Ser eficiente, funcionar en tiempo linear sobre el número de pixeles de una imágen*. Los métodos de segmentación deben funcionar a la par con los métodos de detección de borde u otros algoritmos de bajo nivel.


#### Métodos existentes (conc)
En la época en que se escribió esta investigación, se analizaron otras técnicas existentes. El documento explica como los procesos basados en vectores propios son muy lentos para aplicaciones prácticas. Adicionalmente se menciona que otros métodos, si eficientes, fallan en capturar las propiedades globales (no-locales) de la imágen que son perceptiblemente importantes.